# Herschel Stripe 82 DECAM merging

Both DES and DECaLS provide DECam fluxes which have overlapping coverage. We chose which to use DES preferentially. In this notebook we cross match both catalogues and take the DES fluxes where available, using DECaLS otherwise

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

In [ ]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [ ]:
des = Table.read("{}/des.fits".format(TMP_DIR))
decals = Table.read("{}/DECALS.fits".format(TMP_DIR))

In [ ]:
for col in des.colnames:
    if '_decam_' in col:
        des[col].name = col.replace('_decam_', '_des-decam_')
        
for col in decals.colnames:
    if '_decam_' in col:
        decals[col].name = col.replace('_decam_', '_decals-decam_')

## II - Merging tables

We first merge the optical catalogues and then add the infrared ones: HSC, VHS, VICS82, UKIDSS-LAS, PanSTARRS, SHELA, SpIES.

At every step, we look at the distribution of the distances to the nearest source in the merged catalogue to determine the best crossmatching radius.

### DES

In [ ]:
master_catalogue = des
master_catalogue['des_ra'].name = 'ra'
master_catalogue['des_dec'].name = 'dec'


## Add DECaLS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(decals['decals_ra'], decals['decals_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, decals, "decals_ra", "decals_dec", radius=0.8*u.arcsec)


### Cleaning

When we merge the catalogues, astropy masks the non-existent values (e.g. when a row comes only from a catalogue and has no counterparts in the other, the columns from the latest are masked for that row). We indicate to use NaN for masked values for floats columns, False for flag columns and -1 for ID columns.

In [ ]:
for col in master_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        master_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        master_catalogue[col].fill_value = 0
    elif "id" in col:
        master_catalogue[col].fill_value = -1
        
master_catalogue = master_catalogue.filled()

In [ ]:
master_catalogue[:10].show_in_notebook()

## III - Merging flags and stellarity

Each pristine catalogue contains a flag indicating if the source was associated to a another nearby source that was removed during the cleaning process.  We merge these flags in a single one.

In [ ]:
flag_cleaned_columns = [column for column in master_catalogue.colnames
                        if 'flag_cleaned' in column]

flag_column = np.zeros(len(master_catalogue), dtype=bool)
for column in flag_cleaned_columns:
    flag_column |= master_catalogue[column]
    
master_catalogue.add_column(Column(data=flag_column, name="decam_flag_cleaned"))
master_catalogue.remove_columns(flag_cleaned_columns)

Each pristine catalogue contains a flag indicating the probability of a source being a Gaia object (0: not a Gaia object, 1: possibly, 2: probably, 3: definitely).  We merge these flags taking the highest value.

In [ ]:
flag_gaia_columns = [column for column in master_catalogue.colnames
                     if 'flag_gaia' in column]

master_catalogue.add_column(Column(
    data=np.max([master_catalogue[column] for column in flag_gaia_columns], axis=0),
    name="decam_flag_gaia"
))
master_catalogue.remove_columns(flag_gaia_columns)

Each prisitine catalogue may contain one or several stellarity columns indicating the probability (0 to 1) of each source being a star.  We merge these columns taking the highest value.

In [ ]:
stellarity_columns = [column for column in master_catalogue.colnames
                      if 'stellarity' in column]

master_catalogue.add_column(Column(
    data=np.nanmax([master_catalogue[column] for column in stellarity_columns], axis=0),
    name="decam_stellarity"
))
master_catalogue.remove_columns(stellarity_columns)

## VIII - Cross-identification table

We are producing a table associating to each HELP identifier, the identifiers of the sources in the pristine catalogue. This can be used to easily get additional information from them.

In [ ]:
master_catalogue.add_column(Column(data=(np.char.array(master_catalogue['des_id'].astype(str)) 
                                    +  np.char.array(master_catalogue['decals_id'].astype(str) )), 
                              name="decam_intid"))


In [ ]:

id_names = []
for col in master_catalogue.colnames:
    if '_id' in col:
        id_names += [col]
    if '_intid' in col:
        id_names += [col]
        
print(id_names)

## VI - Choosing between multiple values for the same filter



In [ ]:
decam_origin = Table()
decam_origin.add_column(master_catalogue['decam_intid'])

In [ ]:
decam_stats = Table()
decam_stats.add_column(Column(data=['g','r','i','z','y'], name="Band"))
for col in ["DES", "DECaLS"]:
    decam_stats.add_column(Column(data=np.full(5, 0), name="{}".format(col)))
    decam_stats.add_column(Column(data=np.full(5, 0), name="use {}".format(col)))
    decam_stats.add_column(Column(data=np.full(5, 0), name="{} ap".format(col)))
    decam_stats.add_column(Column(data=np.full(5, 0), name="use {} ap".format(col)))

In [ ]:
decam_bands = ['g','r','i','z','y'] # Lowercase naming convention (k is Ks)
for band in decam_bands:
    if (band == 'i') or (band == 'y'):
        master_catalogue["f_des-decam_{}".format(band)].name = "f_decam_{}".format(band)
        master_catalogue["ferr_des-decam_{}".format(band)].name = "ferr_decam_{}".format(band)
        master_catalogue["m_des-decam_{}".format(band)].name = "m_decam_{}".format(band)
        master_catalogue["merr_des-decam_{}".format(band)].name = "merr_decam_{}".format(band)
        master_catalogue["f_ap_des-decam_{}".format(band)].name = "f_ap_decam_{}".format(band)
        master_catalogue["ferr_ap_des-decam_{}".format(band)].name = "ferr_ap_decam_{}".format(band)
        master_catalogue["m_ap_des-decam_{}".format(band)].name = "m_ap_decam_{}".format(band)
        master_catalogue["merr_ap_des-decam_{}".format(band)].name = "merr_ap_decam_{}".format(band)
        master_catalogue["flag_des-decam_{}".format(band)].name = "flag_decam_{}".format(band)
        
        continue

    # DECam total flux 
    has_des = ~np.isnan(master_catalogue['f_des-decam_' + band])
    has_decals = ~np.isnan(master_catalogue['f_decals-decam_' + band])

    use_des = has_des 
    use_decals = has_decals & ~has_des


    f_decam = np.full(len(master_catalogue), np.nan)
    f_decam[use_des] = master_catalogue['f_des-decam_' + band][use_des]
    f_decam[use_decals] = master_catalogue['f_decals-decam_' + band][use_decals]

    ferr_decam = np.full(len(master_catalogue), np.nan)
    ferr_decam[use_des] = master_catalogue['ferr_des-decam_' + band][use_des]
    ferr_decam[use_decals] = master_catalogue['ferr_decals-decam_' + band][use_decals]
    
    m_decam = np.full(len(master_catalogue), np.nan)
    m_decam[use_des] = master_catalogue['m_des-decam_' + band][use_des]
    m_decam[use_decals] = master_catalogue['m_decals-decam_' + band][use_decals]
    
    merr_decam = np.full(len(master_catalogue), np.nan)
    merr_decam[use_des] = master_catalogue['merr_des-decam_' + band][use_des]
    merr_decam[use_decals] = master_catalogue['merr_decals-decam_' + band][use_decals]
    
    flag_decam = np.full(len(master_catalogue), False, dtype=bool)
    flag_decam[use_des] = master_catalogue['flag_des-decam_' + band][use_des]
    flag_decam[use_decals] = master_catalogue['flag_decals-decam_' + band][use_decals]

    master_catalogue.add_column(Column(data=f_decam, name="f_decam_" + band))
    master_catalogue.add_column(Column(data=ferr_decam, name="ferr_decam_" + band))
    master_catalogue.add_column(Column(data=m_decam, name="m_decam_" + band))
    master_catalogue.add_column(Column(data=merr_decam, name="merr_decam_" + band))
    master_catalogue.add_column(Column(data=flag_decam, name="flag_decam_" + band))


    old_des_columns = ['f_des-decam_' + band,
                               'ferr_des-decam_' + band,
                               'm_des-decam_' + band, 
                               'merr_des-decam_' + band,
                               'flag_des-decam_' + band]
    old_decals_columns =      ['f_decals-decam_' + band,
                               'ferr_decals-decam_' + band,
                               'm_decals-decam_' + band, 
                               'merr_decals-decam_' + band,
                               'flag_decals-decam_' + band]
    
    old_columns = old_des_columns + old_decals_columns
    master_catalogue.remove_columns(old_columns)

    origin = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin[use_des] = "DES"
    origin[use_decals] = "DECaLS"

    
    decam_origin.add_column(Column(data=origin, name= 'f_decam_' + band ))
    
    # DECam aperture flux 
 
    has_ap_des = ~np.isnan(master_catalogue['f_ap_des-decam_' + band])
    has_ap_decals = ~np.isnan(master_catalogue['f_ap_decals-decam_' + band])

    use_ap_des = has_ap_des 
    use_ap_decals = has_ap_decals & ~has_ap_des


    f_ap_decam = np.full(len(master_catalogue), np.nan)
    f_ap_decam[use_ap_des] = master_catalogue['f_ap_des-decam_' + band][use_ap_des]
    f_ap_decam[use_ap_decals] = master_catalogue['f_ap_decals-decam_' + band][use_ap_decals]

    ferr_ap_decam = np.full(len(master_catalogue), np.nan)
    ferr_ap_decam[use_ap_des] = master_catalogue['ferr_ap_des-decam_' + band][use_ap_des]
    ferr_ap_decam[use_ap_decals] = master_catalogue['ferr_ap_decals-decam_' + band][use_ap_decals]
    
    m_ap_decam = np.full(len(master_catalogue), np.nan)
    m_ap_decam[use_ap_des] = master_catalogue['m_ap_des-decam_' + band][use_ap_des]
    m_ap_decam[use_ap_decals] = master_catalogue['m_ap_decals-decam_' + band][use_ap_decals]
    
    merr_ap_decam = np.full(len(master_catalogue), np.nan)
    merr_ap_decam[use_ap_des] = master_catalogue['merr_ap_des-decam_' + band][use_ap_des]
    merr_ap_decam[use_ap_decals] = master_catalogue['merr_ap_decals-decam_' + band][use_ap_decals]
    

    master_catalogue.add_column(Column(data=f_ap_decam, name="f_ap_decam_" + band))
    master_catalogue.add_column(Column(data=ferr_ap_decam, name="ferr_ap_decam_" + band))
    master_catalogue.add_column(Column(data=m_ap_decam, name="m_ap_decam_" + band))
    master_catalogue.add_column(Column(data=merr_ap_decam, name="merr_ap_decam_" + band))
  


    old_ap_des_columns = ['f_ap_des-decam_' + band,
                               'ferr_ap_des-decam_' + band,
                               'm_ap_des-decam_' + band, 
                               'merr_ap_des-decam_' + band]
    old_ap_decals_columns =      ['f_ap_decals-decam_' + band,
                               'ferr_ap_decals-decam_' + band,
                               'm_ap_decals-decam_' + band, 
                               'merr_ap_decals-decam_' + band]
    
    old_ap_columns = old_ap_des_columns + old_ap_decals_columns
    master_catalogue.remove_columns(old_ap_columns)

    origin_ap = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin_ap[use_ap_des] = "DES"
    origin_ap[use_ap_decals] = "DECaLS"

    
    decam_origin.add_column(Column(data=origin_ap, name= 'f_ap_decam_' + band ))    

   
    decam_stats['DES'][decam_stats['Band'] == band] = np.sum(has_des)
    decam_stats['DECaLS'][decam_stats['Band'] == band] = np.sum(has_decals)

    decam_stats['use DES'][decam_stats['Band'] == band] = np.sum(use_des)
    decam_stats['use DECaLS'][decam_stats['Band'] == band] = np.sum(use_decals)

    decam_stats['DES ap'][decam_stats['Band'] == band] = np.sum(has_ap_des)
    decam_stats['DECaLS ap'][decam_stats['Band'] == band] = np.sum(has_ap_decals)

    decam_stats['use DES ap'][decam_stats['Band'] == band] = np.sum(use_ap_des)
    decam_stats['use DECaLS ap'][decam_stats['Band'] == band] = np.sum(use_ap_decals)


In [ ]:
decam_stats.show_in_notebook()

In [ ]:
decam_origin.write("{}/herschel-stripe-82_decam_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

## IX - Saving the catalogue

In [ ]:
master_catalogue.colnames

In [ ]:
master_catalogue["ra"].name = "decam_ra"
master_catalogue["dec"].name = "decam_dec"
master_catalogue["flag_merged"].name = "decam_flag_merged"

In [ ]:
columns = ["decam_intid", "des_id", "decals_id",
           'decam_ra', 'decam_dec', 'decam_flag_merged',  
           'decam_flag_cleaned',  'decam_flag_gaia', 'decam_stellarity']

bands = [column[5:] for column in master_catalogue.colnames if 'f_ap' in column]
for band in bands:
    columns += ["f_ap_{}".format(band), "ferr_ap_{}".format(band),
                "m_ap_{}".format(band), "merr_ap_{}".format(band),
                "f_{}".format(band), "ferr_{}".format(band),
                "m_{}".format(band), "merr_{}".format(band),
                "flag_{}".format(band)]    


In [ ]:
# We check for columns in the master catalogue that we will not save to disk.
print("Missing columns: {}".format(set(master_catalogue.colnames) - set(columns)))

In [ ]:
master_catalogue[:10].show_in_notebook()

In [ ]:
master_catalogue[columns].write("{}/decam_merged_catalogue_herschel-stripe-82.fits".format(TMP_DIR), overwrite=True)